# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096909


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:48,  3.76s/it]

Rendering models:  10%|▉         | 3/31 [00:08<01:22,  2.93s/it]

Rendering models:  13%|█▎        | 4/31 [00:09<01:01,  2.28s/it]

Rendering models:  16%|█▌        | 5/31 [00:10<00:47,  1.83s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:25<00:02,  1.90it/s]

Rendering models:  94%|█████████▎| 29/31 [00:26<00:01,  1.91it/s]

Rendering models:  97%|█████████▋| 30/31 [00:28<00:00,  1.21it/s]

Rendering models: 100%|██████████| 31/31 [00:28<00:00,  1.28it/s]

not-logged-in-3b0b929f4475fd276561    0.000140
not-logged-in-0687f3923f81ac7bd40a    0.005107
Jnursssmith5263                       0.016242
not-logged-in-c80a0b0f46b8cf5098c1    0.000561
not-logged-in-c3e635069550aa98e5fd    0.005762
not-logged-in-8cf2e6929490167ca6ab    0.007650
jnarayanbvg                           0.000201
not-logged-in-c6917d9b19f41e6c2995    0.008698
thiagoxxxx                            0.000250
CScampstudent                         0.000989
Galaxy-one                            0.000317
not-logged-in-6c3c85931d83a1164a0d    0.004387
sn346808                              0.000429
not-logged-in-f7e2260ed4a9517a3e4a    0.012261
mami2019                              0.000307
puic                                  0.000217
amateusz                              0.000580
gadam8                                0.000308
not-logged-in-2105317ef1ec85b34132    0.004018
gadam8                                0.000341
not-logged-in-ca0e5c102eb9a9d65ed3    0.013449
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())